# Fast IP Range Assessment
- Author: Aidan Marlin
- Date: 20220715
- Version: 1.5
- Description: Jupyter Notebook to automate the execution of key infrastructure scans

In [ ]:
import socket
import os

###
### Change these variables if necessary
###
### Specify range/IP like 192.168.0.0/24 or 192.168.0.1/32
ip_range = "192.168.0.0/24"
vpn = True

# Check this before running rest of notebook
router_ip = os.popen('ip r | grep default | choose 2').read().rstrip()
router_mac = os.popen("arp -n | grep \"" + router_ip + " \" | choose 2").read().rstrip()

if vpn == True:
    interface = os.popen('nmcli connection show --active | grep -E "tun|tap|wireguard" | choose 0').read().rstrip()
else:
    interface = os.popen('ip r | grep default | choose 4').read().rstrip()

###
### No need to modify from here
###

os.environ["PATH"] = os.environ["PATH"] + ":/run/current-system/sw/bin"

!mkdir -p tools/masscan
!mkdir -p tools/nmap
!mkdir -p tools/puppeteer

def read_file(location):
    try:
        with open(location) as f:
            text = f.read()
            return text
    except:
        return False

# Local network
print("Router IP: " + router_ip)
print("Router MAC: " + router_mac)
print("Interface: " + interface)


# Target

In [ ]:
print(ip_range)


# Pentester Details

## IP Address

In [ ]:
!curl https://ifconfig.me/


## Start Time

In [ ]:
!date


## Interface Info

In [ ]:
!ip a


# Discovery

## Ping Scan

In [ ]:
!time nmap -sn -T5 {ip_range} -v0 -oA tools/nmap/internal_ping_scan


In [ ]:
print(read_file("tools/nmap/internal_ping_scan.gnmap"))


## Fast TCP Port Scan

In [ ]:
!sudo time masscan --top-ports 100 --interface {interface} --rate=1000 -oB tools/masscan/tcp_top_100.bin {ip_range}


In [ ]:
!sudo masscan --readscan tools/masscan/tcp_top_100.bin -oG tools/masscan/tcp_top_100.grep
!sudo masscan --readscan tools/masscan/tcp_top_100.bin -oJ tools/masscan/tcp_top_100.json
!sudo masscan --readscan tools/masscan/tcp_top_100.bin -oX tools/masscan/tcp_top_100.xml


In [ ]:
print(read_file("tools/masscan/tcp_top_100.grep"))


## Fast SCTP

In [ ]:
!sudo time nmap -T 4 -sY -sV -sC -F -oA tools/nmap/sctp_fast {ip_range}


In [ ]:
print(read_file("tools/nmap/sctp_fast.grep"))


## Fast UDP Port Scan

In [ ]:
!sudo time nmap -T5 -sU --top-ports 100 -v0 {ip_range} -oA tools/nmap/udp_top_100


In [ ]:
print(read_file("tools/nmap/udp_top_100.gnmap"))


## ivre

In [ ]:
%%bash
sed -i "s#- \"80:80\"#- \"127.0.0.1:8010:80\"#g" ${HOME}/git/pentest-tools/ivre/docker/docker-compose.yml
docker-compose -f /home/user/git/pentest-tools/ivre/docker/docker-compose.yml up -d 2>&1 > /dev/null


In [ ]:
ivre_container_id = os.popen("docker ps -f 'name=ivreclient' --format '{{.ID}}'").read().rstrip()
print(ivre_container_id)


In [ ]:
!time docker exec -t $ivre_container_id ivre runscans --network $ip_range 2>&1 > /dev/null


In [ ]:
!time docker exec -t $ivre_container_id ivre getwebdata


In [ ]:
!time docker exec -t $ivre_container_id ivre scan2db -r /scans


In [ ]:
!time docker exec -t $ivre_container_id ivre db2view


Results at http://localhost:8010/

## Searchsploit

In [ ]:
!time docker run --rm -v "$PWD:/mnt" booyaabes/kali-linux-full searchsploit -v --nmap /mnt/tools/nmap/udp_top_100.xml 2>/dev/null


## AutoPWN-Suite

In [ ]:
nist_api_key = read_file("/etc/api-nist")
nist_api_key = nist_api_key.rstrip()

if nist_api_key == False:
    nist_api_key = 0

!echo sudo time python /home/user/git/pentest-tools/AutoPWN-Suite/autopwn.py -y -a {nist_api_key} -t {ip_range}


## Notify

In [ ]:
!/home/user/git/maxos/scripts/telegram_notify.sh -a -m "PT Fast IP Range complete for {ip_range}."


# End Time

In [ ]:
!date
